# Conexión y Extracción de datos desde una API pública de Finanzas

In [77]:
import requests
import json
import os
from dotenv import load_dotenv

def download_data(api_url,params):
    response = requests.get(api_url,params=params)
    
    if response.status_code == 200:
        try:
            data = json.loads(response.text)
            errors = [(value['code'],value['message'],value['status']) for key, value in data.items() if 'code' in value and value['code'] in [400, 401, 403, 404, 414, 429, 500]]
            if errors:
                for error in errors:
                    code, message, status = error
                    print(f"{code}\n{message}\n{status}\n")
                return None
            else:
                print("Los datos se han extraído correctamente!")
                return data
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
    else:
        print(f"Request failed with status code: {response.status_code}")
    
    return None

load_dotenv()

base_url = 'https://api.twelvedata.com' 
endpoint = '/time_series' 
params = {
    'symbol': 'AAPL,AMZN,TSLA,META,MSFT,GOOG,SPY,QQQ',
    'interval': '1day',
    'start_date': '2024-01-01',
    'apikey': os.getenv('APIKEY')
}

api_url = base_url + endpoint

data = download_data(api_url,params)

Los datos se han extraído correctamente!


# Obtención del DataFrame y Transformación de datos

In [78]:
import pandas as pd
from datetime import datetime

# Transformación de datos
for key, value in data.items():
    for val in value['values']:
        if 'datetime' in val:
            val['datetime'] = datetime.strptime(val['datetime'], "%Y-%m-%d")
        if 'open' in val:
            val['open'] = round(float(val['open']), 3)
        if 'high' in val:
            val['high'] = round(float(val['high']), 3)
        if 'low' in val:
            val['low'] = round(float(val['low']), 3)
        if 'close' in val:
            val['close'] = round(float(val['close']), 3)
        if 'volume' in val:
            val['volume'] = int(val['volume'])

# Creación del dataframe a partir de una lista de compresión
df = pd.DataFrame([
    {
        'symbol': value['meta']['symbol'],
        'currency': value['meta']['currency'],
        'exchange_timezone': value['meta']['exchange_timezone'],
        'exchange': value['meta']['exchange'],
        'mic_code': value['meta']['mic_code'],
        'type': value['meta']['type'],
        **val,
        'datetime_load': pd.Timestamp.now().round("1s"),
    }
    for key, value in data.items() 
        for val in value['values']
])

df = df.rename(columns={'open': 'open_value', 'high': 'high_value', 'low': 'low_value', 'close': 'close_value'})

df.head()

,symbol,currency,exchange_timezone,exchange,mic_code,type,datetime,open_value,high_value,low_value,close_value,volume,datetime_load
0,AAPL,USD,America/New_York,NASDAQ,XNGS,Common Stock,2024-04-29,173.39,176.03,173.12,173.50,66144936,2024-04-29 20:27:19
1,AAPL,USD,America/New_York,NASDAQ,XNGS,Common Stock,2024-04-26,169.88,171.34,169.18,169.30,44525100,2024-04-29 20:27:19
2,AAPL,USD,America/New_York,NASDAQ,XNGS,Common Stock,2024-04-25,169.53,170.61,168.15,169.89,50558300,2024-04-29 20:27:19
3,AAPL,USD,America/New_York,NASDAQ,XNGS,Common Stock,2024-04-24,166.54,169.30,166.21,169.02,48251800,2024-04-29 20:27:19
4,AAPL,USD,America/New_York,NASDAQ,XNGS,Common Stock,2024-04-23,165.35,167.05,164.92,166.90,49537800,2024-04-29 20:27:19


# Conexión a la Base de Datos en Amazon Redshift

In [79]:
import psycopg2

load_dotenv()

try:
    conn = psycopg2.connect(
        host = os.getenv('AR_HOST'),
        dbname = os.getenv('AR_DATABASE'),
        user = os.getenv('AR_USER'),
        password = os.getenv('AR_PASSWORD'),
        port = os.getenv('AR_PORT')
    )
    print("Se ha establecido la conexión con Amazon Redshift de manera exitosa!\n")
except Exception as e:
    print(f"No es posible establecer la conexión con Amazon Redshift\nError: {e}\n")

Se ha establecido la conexión con Amazon Redshift de manera exitosa!



# Creación de Tabla

In [80]:
table_name = 'serenadituro_coderhouse.finances'
try:
    with conn.cursor() as cur:
        create_table = f''' CREATE TABLE IF NOT EXISTS {table_name} (
                    symbol VARCHAR(10) NOT NULL,
                    currency VARCHAR(30) NOT NULL,
                    exchange_timezone VARCHAR(50) NOT NULL,
                    exchange VARCHAR(20) NOT NULL,
                    mic_code VARCHAR(10) NOT NULL,
                    type VARCHAR(30) NOT NULL,
                    datetime DATE NOT NULL,
                    open_value FLOAT NOT NULL,
                    high_value FLOAT NOT NULL,
                    low_value FLOAT NOT NULL,
                    close_value FLOAT NOT NULL,
                    volume INT NOT NULL,
                    datetime_load DATETIME NOT NULL,
                    PRIMARY KEY(symbol,datetime)
                )'''
        cur.execute(create_table)
        conn.commit()
        print(f'Operación realizada con éxito!\n')
except Exception as e:
    print(f'Error al crear la tabla\n{e}\n')

Operación realizada con éxito!



# Carga de datos a la tabla de Redshift

In [81]:
try: 
    with conn.cursor() as cur:
        index = 0
        for row in df.itertuples(index=False):
            # verifico si el dato fue previamente insertado en la tabla
            select_data = f'''SELECT COUNT(*) FROM {table_name} WHERE symbol = %s AND datetime = %s'''
            cur.execute(select_data, (row.symbol, row.datetime))
            result = cur.fetchone()
            # si no existe un dato con el mismo símbolo y fecha asociada, se inserta en la tabla
            if result[0] == 0:  
                placeholder = ', '.join(['%s'] * len(df.columns))
                insert_data = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({placeholder})"
                cur.execute(insert_data, df.values.tolist()[index])
                print('Dato insertado...\n')
            index += 1
    conn.commit()
    print(f'Los datos se han insertado correctamente!\n')
except Exception as e:
    print(f'Error al insertar los datos\n{e}\n')

Los datos se han insertado correctamente!



# Cierre de la conexión a la Base de Datos

In [82]:
conn.close()